In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Oct 19 14:23:54 2024
@author: ishiv
"""
import requests
import json
from bs4 import BeautifulSoup
import lxml
import time
import csv
import random
import re
import os
import pandas as pd

USERNAME = 'ShivangBaijal'
PASS = 'aMBTxo'

#USRT_PAGE=0
MAXERNAME = 'ShivangBaijal'
#PASS = 'aMBTxo'
MAX_PAGES_TO_BROWSE= 500
#STA_PAGES_TO_BROWSE= 200
MAX_DELAY= 3 # Maximum Delay after each request
MIN_DELAY=1
#USERNAME = 'ShivangBaijal'
#PASS = 'aMBTxo'

loginurl = ('https://www.valueinvestorsclub.com/login')
# loginurl = (f'https://api.zenrows.com/v1/?apikey=4a0367174080c0b42f626705383fd7784b6034b3&url={loginurl}&session_id=20')
search_url=('https://valueinvestorsclub.com/messages/loadmsgs')
# search_url=(f"(f'https://api.zenrows.com/v1/?apikey=4a0367174080c0b42f626705383fd7784b6034b3&url={search_url}&session_id=20')")
payload_url=('https://valueinvestorsclub.com/messages/loadmsgs')
# payload_url = (f"(f'https://api.zenrows.com/v1/?apikey=4a0367174080c0b42f626705383fd7784b6034b3&url={payload_url}&session_id=20')")
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36'}
payload_login ={
	  'forum':'',
	  'code':'',
	  'login[login_name]':USERNAME,
	  'login[password]':PASS,
	  'commit': 'Login'}

results =[]
result_dict = {}
csv_data=[]

def request_session(loginurl, headers, payload_login):
	with requests.Session() as s:
		r = s.get(loginurl,headers=headers)
		soup=BeautifulSoup(r.content,"lxml")
		payload_login['_token']=soup.find('input',attrs={'name':'_token'})['value']
		r = s.post(loginurl, data=payload_login,headers=headers)
		return s


def get_original_author(url, req_session, headers):
    try:
        # r_url = (f'https://api.zenrows.com/v1/?apikey=4a0367174080c0b42f626705383fd7784b6034b3&url={url}&session_id=20')
        #time.sleep(random.randint(MIN_DELAY*100,MAX_DELAY*100)/100)
        print(f"Fetching Original idea from: {url}")
        web_page = req_session.get(url, headers=headers)
        print(f"Status code: {web_page.status_code}")
        original_idea_by = BeautifulSoup(web_page.text,"lxml").select_one('.idea_by a.display_name').get('title')
        print(f"original idea author: {original_idea_by}")
        return original_idea_by
    except:
        with open(f"response_{url.rsplit('/')[0]}.txt", "w") as f:
            f.write(web_page.text)
        return ""


s=request_session(loginurl, headers, payload_login)
os.chdir("/Users/shivangbaijal/Library/CloudStorage/GoogleDrive-shivang.baijal@pluralinvesting.com/Shared drives/Plural Investing LLC/Data Projects/VIC/VIC Phase 3 - Development/Scrapped Data/")

for page_no in range(0,MAX_PAGES_TO_BROWSE+1):
	payload = {'type': 'all',
				'page': str(page_no),
				'sort': 'new',
				'shorten_name': '0'}
	#time.sleep(random.randint(20,MAX_DELAY*50)/100)
	response = s.post(search_url,data=payload,headers=headers)
	print(response)
	try:
		results.extend(response.json()['result'])
		print(f"Found {len(response.json()['result'])} messages on page {str(page_no)}")
	except:
		pass

print(f"\n--- Found {len(results)} items ---")
	#time.sleep(random.randint(MIN_DELAY*10,MAX_DELAY*10)/100)
# csv_data

<Response [200]>
Found 20 messages on page 0
<Response [200]>
Found 20 messages on page 1
<Response [200]>
Found 20 messages on page 2
<Response [200]>
Found 20 messages on page 3
<Response [200]>
Found 20 messages on page 4
<Response [200]>
Found 20 messages on page 5
<Response [200]>
Found 20 messages on page 6
<Response [200]>
Found 20 messages on page 7
<Response [200]>
Found 20 messages on page 8
<Response [200]>
Found 20 messages on page 9
<Response [200]>
Found 20 messages on page 10
<Response [200]>
Found 20 messages on page 11
<Response [200]>
Found 20 messages on page 12
<Response [200]>
Found 20 messages on page 13
<Response [200]>
Found 20 messages on page 14
<Response [200]>
Found 20 messages on page 15
<Response [200]>
Found 20 messages on page 16
<Response [200]>
Found 20 messages on page 17
<Response [200]>
Found 20 messages on page 18
<Response [200]>
Found 20 messages on page 19
<Response [200]>
Found 20 messages on page 20
<Response [200]>
Found 20 messages on page 2

In [ ]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from bs4 import BeautifulSoup

# Define a function to process each result
def process_result(result):
    result_dict = {}
    
    # Prepare structured data
    result_dict['id'] = result['id']
    result_dict['user_id'] = result['user_id']
    result_dict['idea_id'] = result['idea_id']
    result_dict['idea_keyid'] = result['idea_keyid']
    result_dict['idea_add_date'] = result['idea_add_date']
    result_dict['m_add_date'] = result['m_add_date']
    result_dict['votes'] = result['votes']
    result_dict['up'] = result['up']
    result_dict['down'] = result['down']
    result_dict['Long_Short'] = 'Long' if result['is_long'] == 1 else 'Short'
    result_dict['Is_Private'] = result['is_private']
    
    # Handle company name and symbol
    company_name = result['thread']
    result_dict['company_name'] = company_name.replace('&amp;', '') if company_name else company_name
    result_dict['company_symbol'] = result['symbol']
    result_dict['display_name'] = result['display_name']
    result_dict['idea_status'] = result['idea_status']
    
    # Generate URL and find original idea author
    result_dict['URL'] = "https://valueinvestorsclub.com" + BeautifulSoup(result['subject_link'], "lxml").find('a')['href']
    result_dict['Original_idea_author'] = get_original_author(result_dict['URL'], req_session=s, headers=headers)
    
    # Clean up message body
    message_body = BeautifulSoup(result['read_more'], "lxml").get_text()
    result_dict['message_body'] = message_body.replace('&nbsp;', ' ') if message_body else message_body
    result_dict['Subject'] = result['subject']
    
    return result_dict

# Using ThreadPoolExecutor to process results in parallel
csv_data = []
with ThreadPoolExecutor(max_workers=80) as executor:
    futures = {executor.submit(process_result, result): result for result in results}
    for i, future in enumerate(as_completed(futures)):
        result_dict = future.result()
        csv_data.append(result_dict)
        print(f"Processed {i+1}/{len(results)}")


Fetching Original idea from: https://valueinvestorsclub.com/idea/SoftwareOne/2314605997/messages/225357Fetching Original idea from: https://valueinvestorsclub.com/idea/Kaspi.kz/8547833282/messages/225350
Fetching Original idea from: https://valueinvestorsclub.com/idea/TT_Electronics/6934012669/messages/225356

Fetching Original idea from: https://valueinvestorsclub.com/idea/Gulf_Marine_Services_PLC/2766381048/messages/225349
Fetching Original idea from: https://valueinvestorsclub.com/idea/INMUNE_BIO_INC/3528803511/messages/225348
Fetching Original idea from: https://valueinvestorsclub.com/idea/INMUNE_BIO_INC/3528803511/messages/225345
Fetching Original idea from: https://valueinvestorsclub.com/idea/IMPERIAL_PETE_INC/9691784942/messages/225344
Fetching Original idea from: https://valueinvestorsclub.com/idea/IMMERSION_CORP/4034416218/messages/225342
Fetching Original idea from: https://valueinvestorsclub.com/idea/IMPERIAL_PETE_INC/9691784942/messages/225339
Fetching Original idea from: h

In [21]:
with open(f'VIC_Messages_Updated1_test.csv','w+',newline='',encoding='utf-8') as message_file:
			dictwriter=csv.DictWriter(message_file,fieldnames=list(csv_data[0].keys()))
			dictwriter.writeheader()
			dictwriter.writerows(csv_data)
			message_file.close()

print("\nData saved in CSV file")
#for r in data.columns:
#    data[r]=data[r].astype(str)
new_data=pd.read_csv('VIC_Messages_Updated1_test.csv')

list(new_data.columns)
new_data=new_data[['id','company_name','company_symbol','Original_idea_author','idea_add_date',
          'display_name','m_add_date','votes','up','down','Long_Short',
          'Is_Private','URL','Subject','message_body']].copy()

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/shivangbaijal/Library/CloudStorage/GoogleDrive-shivang.baijal@pluralinvesting.com/Shared drives/Plural Investing LLC/Data Projects/VIC/VIC Phase 2 - Production/vic-project-400107-4bfdc9864179.json"
import pandas as pd
from google.cloud import bigquery

# Initialize a BigQuery client
client = bigquery.Client()
df = pd.DataFrame(new_data)

# Specify your dataset and table name
dataset_id = 'VIC'
table_id = 'VIC_Clean_Messages'
table_ref = client.dataset(dataset_id).table(table_id)

#Job configuration for appending (Comment these line for first time append)
job_config = bigquery.LoadJobConfig()
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND 

# Push the DataFrame to BigQuery for appending
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()  # Wait for the job to complete
print(f"Appended {job.output_rows} rows to {dataset_id}.{table_id}.")


Data saved in CSV file
Appended 2019 rows to VIC.VIC_Clean_Messages.
